To-Do
- Save to pickle and then second notebook for data preparation
- Specify procedure for outliers
- Binary loan grant variable for classification (and drop of action taken)
- Check standardization of features
- Interaction Terms for protected attributes

Category: conforming_loan_limit; action_taken; purchaser_type; loan_type; loan_purpose; lien_status; reverse_mortgage; open-end_line_of_credit; business_or_commercial_purpose (check and remove commercial); hoepa_status; negative_amortization; interest_only_payment; balloon_payment; other_nonamortizing_features; occupancy_type; manufactured_home_secured_properly; manufactured_home_land_property_interest; total_units; applicant_credit_score_type; applicant_ethnicity-1; applicant_ethnicity-2; applicant_ethnicity_observed; applicant_race-1; applicant_race-2; applicant_race_observed; applicant_sex; applicant_sex_observed; submission_of_application; initially_payable_to_institution;aus-1

# Data Cleaning and EDA - Continued

This notebook picks up where the first analysis notebook has ended, after value imputation and preliminary outlier removal. The pickle saved at the end of the first workbook will be the starting point here.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid", context= "paper")

In [3]:
HMDA_clean = pd.read_pickle("C:/Users/Hauke/OneDrive - ucp.pt/04_Thesis/00_GitHub/Thesis/data/processed/HMDA_cleaned.pkl")

In [6]:
HMDA_clean.head()

,county_code,conforming_loan_limit,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,...,applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race_observed,applicant_sex,applicant_sex_observed,applicant_age,submission_of_application,initially_payable_to_institution,aus-1
1,6065.0,0.0,6,0,2,2,5,1,2,2,...,3,7.0,99.0,3,4,3,8888.0,3,3,6
2,6065.0,0.0,6,0,2,2,1,1,2,2,...,3,7.0,99.0,3,4,3,8888.0,3,3,6
3,6099.0,0.0,1,1,2,1,1,1,2,2,...,2,5.0,99.0,2,1,2,3.0,1,1,1
4,6111.0,1.0,3,0,2,1,31,1,2,2,...,2,6.0,99.0,2,1,2,4.0,1,1,6
5,6019.0,0.0,4,0,2,1,32,1,2,2,...,2,6.0,99.0,2,2,2,1.0,1,1,1
